# Feature Engineering

## 데이터 전처리 및 특성 추출

In [1]:
import pandas as pd

In [7]:
# 훈련 데이터와 테스트 데이터 합치기
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

train_test_data = [train, test]

### Name Feature

In [19]:
# ' ([A-Za-z]+)\.' : 공백으로 시작하고 .(점)으로 끝나는 문자열
for data in train_test_data:
    data['Title'] = data.Name.str.extract(' ([A-Za-z]+)\.')

train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr


In [22]:
# 추출한 Title을 가진 사람의 수를 출력(성별에 따른)
# Pandas crosstab을 이용해서 빈도표 만들기
pd.crosstab(train['Title'], train['Sex'])

Sex,female,male
Title,,
Capt,0,1
Col,0,2
Countess,1,0
Don,0,1
Dr,1,6
Jonkheer,0,1
Lady,1,0
Major,0,2
Master,0,40


In [25]:
for data in train_test_data:
    data['Title'] = data['Title'].replace(['Capt', 'Col', 'Don','Dona', 'Dr', 'Jonkheer', 'Major', 'Rev'], 'Other')
    data['Title'] = data['Title'].replace(['Countess', 'Lady', 'Sir'], 'Royal')
    data['Title'] = data['Title'].replace('Mlle', 'Miss')
    data['Title'] = data['Title'].replace('Mme', 'Mrs')
    data['Title'] = data['Title'].replace('Ms', 'Miss')

In [26]:
pd.crosstab(train['Title'], train['Sex'])

Sex,female,male
Title,,
Master,0,40
Miss,185,0
Mr,0,517
Mrs,126,0
Other,1,19
Royal,2,1


In [28]:
# Title을 숫자로 매핑하기
title_mapping = {
    'Master': 1, 'Miss': 2, 'Mr': 3,
    'Mrs': 4, 'Other': 5, 'Royal':6
}

for data in train_test_data:
    data['Title'] = data['Title'].map(title_mapping)

train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,3
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,4
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,4
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,3
